In [1]:
import torch
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list | head


In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia


In [ ]:
!unzip chest-xray-pneumonia.zip -d data


In [ ]:
import torch
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
train_dir="data/chest_xray/train"
val="data/chest_xray/val"
test_dir="data/chest_xray/test"

train_data=datasets.ImageFolder(train_dir,transform=train_transforms)
val_data=datasets.ImageFolder(val,transform=test_transforms)
test_data=datasets.ImageFolder(test_dir,transform=test_transforms)


In [ ]:
batch_size=32

train_dataloader=DataLoader(train_data,batch_size=batch_size,shuffle=True)
val_dataloader =  DataLoader(val_data,batch_size=batch_size,shuffle=False)
test_dataloader=DataLoader(test_data,batch_size=batch_size,shuffle=False)

In [ ]:
print(f"Number of training batches: {len(train_dataloader)}")
print(f"Number of validation batches: {len(val_dataloader)}")
print(f"Number of test batches: {len(test_dataloader)}")

In [ ]:
#Creating a model for CNN
import torch
from torch import nn
class CNN(nn.Module):
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
    super().__init__()
    self.conv_block_1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )
    self.conv_block_2= nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Dropout(0.3),
        nn.Linear(in_features=hidden_units*14*14,
                  out_features=output_shape)
    )

  def forward(self,x):
      x=self.conv_block_1(x)
      x=self.conv_block_2(x)
      x=self.classifier(x)
      return x

In [ ]:
model=CNN(input_shape=3,hidden_units=16,output_shape=1).to(device)

In [ ]:
loss=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
#Create a train and Test loop function
def train_loop(model, dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss, total_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device).float().unsqueeze(1)

        # Forward pass
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        total_loss += loss.item()

        # Compute accuracy
        preds = torch.sigmoid(y_pred)
        preds = torch.round(preds)
        total_acc += (preds == y).sum().item() / len(y)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)

    print(f"Train Loss: {avg_loss:.4f} | Train Acc: {avg_acc*100:.2f}%")
    return avg_loss, avg_acc


In [ ]:
def test_loop(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device):
    model.eval()
    total_loss, total_acc = 0, 0

    with torch.inference_mode():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device).float().unsqueeze(1)
            y_pred = model(X)

            # Compute loss
            loss = loss_fn(y_pred, y)
            total_loss += loss.item()

            # Compute accuracy (binary)
            preds = torch.sigmoid(y_pred)
            preds = torch.round(preds)
            total_acc += (preds == y).sum().item() / len(y)

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)

    print(f"Test Loss: {avg_loss:.4f} | Test Acc: {avg_acc*100:.2f}%")
    return avg_loss, avg_acc


In [ ]:
#Main loop
epochs=3
train_losses,train_accs=[],[]
test_losses,test_accs=[],[]
for epoch in range(epochs):
  print(f"Epoch: {epoch+1}")
  train_loss,train_acc=train_loop(model=model,dataloader=train_dataloader,loss_fn=loss,optimizer=optimizer,device=device)
  test_loss,test_acc=test_loop(model=model,dataloader=val_dataloader,loss_fn=loss,device=device)
  train_losses.append(train_loss)
  train_accs.append(train_acc)
  test_losses.append(test_loss)
  test_accs.append(test_acc)
  print("-"*50)



# Using ResNet Pretrained Model

In [ ]:
train_transforms_ResNet=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms_ResNet=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_dataset = ImageFolder(root=train_dir, transform=train_transforms)
test_dataset = ImageFolder(root=test_dir, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torchvision.models as models

# Load a pretrained ResNet model (e.g., ResNet18)
model_resnet = models.resnet18(pretrained=True)

# Modify the final layer for binary classification (pneumonia vs. normal)
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = torch.nn.Linear(num_ftrs, 1) # Output shape is 1 for binary classification with BCEWithLogitsLoss

# Move the model to the appropriate device
model_resnet = model_resnet.to(device)

In [ ]:
epochs = 5
train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss, train_acc = train_loop(model, train_loader, loss, optimizer, device)
    val_loss, val_acc = test_loop(model, val_loader, loss, device)

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print("-"*50)


In [ ]:
import matplotlib.pyplot as plt

# Loss
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Accuracy
plt.plot(train_accs, label="Train Accuracy")
plt.plot(val_accs, label="Validation Accuracy")
plt.title("Accuracy Curve")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
torch.save(model_resnet.state_dict(), "resnet18_pneumonia.pt")


In [ ]:
+











